In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 29.2 MB/s eta 0:00:00


In [10]:
import cv2
import mediapipe as mp
import numpy as np
import csv

In [11]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [12]:
def calculate_angle(landmark1, landmark2):
    dy = landmark2[1] - landmark1[1]
    dx = landmark2[0] - landmark1[0]
    angle_radians = np.arctan2(dy, dx)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

In [13]:
def extract_coordinates(results):
    landmarks = results.pose_landmarks.landmark
    coordinates = {
        "left_ear": (landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y),
        "left_shoulder": (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    }
    return coordinates

In [14]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    rows = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            coordinates = extract_coordinates(results)

            # 귀, 어깨 거리 계산
            acromion_distance = abs(coordinates['left_ear'][0] - coordinates['left_shoulder'][0])
            angle = calculate_angle(coordinates['left_ear'], coordinates['left_shoulder'])

            # 거북이 판별
            turtleneck_status = "거북이" if acromion_distance >= 2.5 or angle >= 50 else "정상"
            coordinates['turtleneck_status'] = turtleneck_status
            rows.append(coordinates)

            coordinates['turtleneck_status'] = turtleneck_status
            rows.append(coordinates)

    cap.release()
    save_to_csv(output_path, rows)

In [15]:
def extract_coordinates(results):
    landmarks = results.pose_landmarks.landmark

# left 기준
    coordinates = {
        "left_ear": (landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y),
        "left_shoulder": (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    }

    return coordinates

In [16]:
def save_to_csv(output_path, rows):
    with open(output_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["left_ear_x", "left_ear_y", "left_shoulder_x", "left_shoulder_y", "turtleneck_status"])

        for row in rows:
            writer.writerow([row["left_ear"][0], row["left_ear"][1], row["left_shoulder"][0], row["left_shoulder"][1], row["turtleneck_status"]])

In [18]:
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/GDSC_AI/SC_test/test_video/turtle_neck_test_video.mp4"
    output_path = "turtle_neck_AddAngle.csv"
    process_video(video_path, output_path)

In [19]:
import pandas as pd

df = pd.read_csv('/content/turtle_neck_AddAngle.csv')

df

,left_ear_x,left_ear_y,left_shoulder_x,left_shoulder_y,turtleneck_status
0,0.523811,0.355296,0.569402,0.505713,거북이
1,0.523811,0.355296,0.569402,0.505713,거북이
2,0.523456,0.351931,0.569691,0.506388,거북이
3,0.523456,0.351931,0.569691,0.506388,거북이
4,0.523345,0.350401,0.569818,0.506804,거북이
...,...,...,...,...,...
12573,0.567442,0.355427,0.590571,0.471411,거북이
12574,0.566515,0.357197,0.590917,0.471612,거북이
12575,0.566515,0.357197,0.590917,0.471612,거북이
12576,0.565993,0.358549,0.591128,0.472609,거북이


In [27]:
df.value_counts()

left_ear_x  left_ear_y  left_shoulder_x  left_shoulder_y  turtleneck_status
0.334417    0.111683    0.370302         0.123362         정상                   2
0.543634    0.328409    0.560789         0.421142         거북이                  2
0.543585    0.359096    0.568664         0.462146         거북이                  2
0.543569    0.356451    0.574298         0.469546         거북이                  2
0.543553    0.259129    0.568559         0.370389         거북이                  2
                                                                              ..
0.503624    0.402763    0.493081         0.455099         거북이                  2
0.503611    0.399118    0.494816         0.449379         거북이                  2
0.503609    0.402765    0.493077         0.455094         거북이                  2
            0.401363    0.493778         0.451627         거북이                  2
0.643569    0.165100    0.636960         0.262680         거북이                  2
Length: 6289, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12578 entries, 0 to 12577
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   left_ear_x         12578 non-null  float64
 1   left_ear_y         12578 non-null  float64
 2   left_shoulder_x    12578 non-null  float64
 3   left_shoulder_y    12578 non-null  float64
 4   turtleneck_status  12578 non-null  object 
dtypes: float64(4), object(1)
memory usage: 491.5+ KB


In [28]:
df['turtleneck_status'].value_counts()

거북이    12370
정상       208
Name: turtleneck_status, dtype: int64